In [1]:
## Load packages
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import datetime
import cmocean
import sys
import pyfesom2 as pf
import numpy as np
from scipy.interpolate import griddata
from tqdm import tqdm

from pathlib import Path
#example_data_path = Path("../parcels_example_data/")
import parcels
from matplotlib import pyplot as plt
import cartopy
import datetime

/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/climatology.py:14: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater as sw


In [2]:
def interpolate_at_level_zero(meshpath, str_id=str, year=1958, method=str):
    """
    Interpolates data at level 0 for a given variable over time.

    Parameters:
    - meshpath (str): Path to the directory containing the data files.
    - str_id (str): Variable name to interpolate (e.g., 'v').
    - year (int): Year of the data file to use.
    - method (str): Interpolation method ('linear', 'cubic', or 'nearest').

    Returns:
    - v_grid (np.array): Interpolated data at level 0 over the specified time range.
    """
    alpha, beta, gamma=[0, 0, 0]
    print("mesh will be loaded")
    # Insert your custom path
    meshpath = '/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel'
    mesh = pf.load_mesh(meshpath, abg=[alpha, beta, gamma], usepickle = False)
    #resultpath = f'{meshpath}results/'
    
    print(meshpath)
    
    meshdiag = xr.open_mfdataset(f'{meshpath}/fesom.mesh.diag.nc')
    #print(meshdiag)
    
    # Use the 'elem' dimension size directly
    elem_n = meshdiag.dims["elem"]
    
    # Initialize arrays based on the number of elements #array full of zeros
    xx2 = np.zeros(shape=(elem_n)) 
    yy2 = np.zeros(shape=(elem_n))
    
    #mesh.x2 is the longitude in vector
    #mesh.y2 is the latitude in vector
    
    mesh.x2[mesh.elem[1,:]].mean(axis=0)
    
    #lon=mesh.x2; lat=mesh.y2;
    
    #aa = mesh.elem ---> these are the indices of the elements in the triangle
    ## the loops find the mean of triangle elements or the centroid
    for i in np.arange(0,elem_n):
        xx2[i]=mesh.x2[mesh.elem[i,:]].mean(axis=0)
        yy2[i]=mesh.y2[mesh.elem[i,:]].mean(axis=0)
    
    
    cyclic_length=4.5
    unglue = True
    if unglue:
        try:
            tri = np.loadtxt(f'{meshpath}/elem2d.out', skiprows=1, dtype=int)
            nodes = np.loadtxt(f'{meshpath}/nod2d.out', skiprows=1)
            xcoord, ycoord = nodes[:, 1], nodes[:, 2]
            xc, yc = xcoord[tri - 1], ycoord[tri - 1]
            xmin = xc.min(axis=1)
            for i in range(3):
                ai = np.where(xc[:, i] - xmin > cyclic_length / 2)
                xc[ai, i] -= cyclic_length
            X = xc.mean(axis=1)
            Y = yc.mean(axis=1)
        except FileNotFoundError:
            print("Required files for ungluing not found; skipping unglue step.")
    
    ## the grid
    #Define the target grid with shape (72, 292)
    lon_grid, lat_grid = np.meshgrid(
        np.linspace(X.min(), X.max(), 72),
        np.linspace(Y.min(), Y.max(), 292)
    )
    
    # Flattened original data arrays (21120,)
    X = X.flatten()
    Y = Y.flatten()
    
    # Combine X and Y into a single array of shape (21120, 2) for original coordinates
    coords = np.column_stack((X, Y))
    
    # Flatten the target grid for griddata
    target_coords = np.column_stack((lon_grid.flatten(), lat_grid.flatten()))
    # Load the dataset and select level 0
    dat1 = xr.open_dataset(f'{meshpath}/{str_id}.fesom.{year}.nc')[str_id]
    
    # Initialize output grid
    int_grid = np.zeros((292, 72, dat1.shape[0]))
    #time = np.zeros((dat1.shape[0],))
    # Flattened original data arrays (21120,)
    X = X.flatten()
    Y = Y.flatten()
    coords = np.column_stack((X, Y))  # Combine X and Y for original coordinates

    # Define the target grid
    lon_grid, lat_grid = np.meshgrid(
        np.linspace(X.min(), X.max(), 72),
        np.linspace(Y.min(), Y.max(), 292)
    )
    target_coords = np.column_stack((lon_grid.flatten(), lat_grid.flatten()))

    # Interpolate for each time step at level 0
    #for itime in tqdm(range(dat1.shape[0])):
    for itime in tqdm(range(100)): #try with 100 days

        dat2 = dat1.isel(time=itime, nz1=0).squeeze()  # Select level 0
        data_flat = dat2.values.flatten()

        # Interpolate the data to the target grid
        data_grid_flat = griddata(
            coords,          # Original element coordinates
            data_flat,       # Original data values
            target_coords,   # Target grid coordinates
            method=method    # Interpolation method
        )

        # Reshape interpolated data and assign to output array
        int_grid[:, :, itime] = data_grid_flat.reshape((292, 72))

        # Print date for each time step
        #print(dat1.time.values[itime].astype('datetime64[D]'))
        time = dat1.time.values
        #len(itime.keys())
    return int_grid, lon_grid,lat_grid, time

In [3]:
meshpath = '/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel'

[u_grid_result,lonip,latip,time] = interpolate_at_level_zero(meshpath, str_id='u', year=1960, method='linear')
v_grid_result = interpolate_at_level_zero(meshpath, str_id='v', year=1960, method='linear')[0]


mesh will be loaded
Load 2d part of the mesh in 0 second(s)
/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel


/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/load_mesh_data.py:189: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file_content = pd.read_csv(
/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/load_mesh_data.py:200: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file_content = pd.read_csv(
/tmp/ipykernel_1732329/600224879.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  elem_n = meshdiag.dims["elem"]
100%|█████████████████████████████████████| 100/100 [00:42<00:00,  2.33it/s]


mesh will be loaded
Load 2d part of the mesh in 0 second(s)
/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel


/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/load_mesh_data.py:189: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file_content = pd.read_csv(
/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/load_mesh_data.py:200: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file_content = pd.read_csv(
/tmp/ipykernel_1732329/600224879.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  elem_n = meshdiag.dims["elem"]
100%|█████████████████████████████████████| 100/100 [00:43<00:00,  2.30it/s]


In [4]:
u = np.transpose(u_grid_result, (2, 0, 1))  # Transpose to [time, lat, lon]
v = np.transpose(v_grid_result, (2, 0, 1))  # Transpose to [time, lat, lon]

In [27]:
time_coord = np.arange(0, u.shape[0] * 86400, 86400)  # Adjust based on your data's time intervals

In [28]:
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4

#time_coord=time
# Assuming `lon_grid`, `lat_grid`, and `time_coord` represent the coordinates for your data
lon_grid = lonip  # Replace with actual longitudes
lat_grid = latip   # Replace with actual latitudes

# Define the fields directly from the numpy arrays
data_fields = {
    'U': Field('U', data=u, lon=lon_grid, lat=lat_grid, time=time_coord),
    'V': Field('V', data=v, lon=lon_grid, lat=lat_grid, time=time_coord)
}

fieldset = FieldSet(**data_fields)

In [35]:
time_coord

array([       0,    86400,   172800,   259200,   345600,   432000,
         518400,   604800,   691200,   777600,   864000,   950400,
        1036800,  1123200,  1209600,  1296000,  1382400,  1468800,
        1555200,  1641600,  1728000,  1814400,  1900800,  1987200,
        2073600,  2160000,  2246400,  2332800,  2419200,  2505600,
        2592000,  2678400,  2764800,  2851200,  2937600,  3024000,
        3110400,  3196800,  3283200,  3369600,  3456000,  3542400,
        3628800,  3715200,  3801600,  3888000,  3974400,  4060800,
        4147200,  4233600,  4320000,  4406400,  4492800,  4579200,
        4665600,  4752000,  4838400,  4924800,  5011200,  5097600,
        5184000,  5270400,  5356800,  5443200,  5529600,  5616000,
        5702400,  5788800,  5875200,  5961600,  6048000,  6134400,
        6220800,  6307200,  6393600,  6480000,  6566400,  6652800,
        6739200,  6825600,  6912000,  6998400,  7084800,  7171200,
        7257600,  7344000,  7430400,  7516800,  7603200,  7689

In [30]:
number_particles = 100
## we define the region of release 
lon_bds = np.min(fieldset.U.grid.lon), np.max(fieldset.U.grid.lon)
lat_bds = np.min(fieldset.U.grid.lat), np.max(fieldset.U.grid.lat)

In [31]:
pset = parcels.ParticleSet.from_list(
    fieldset=fieldset,
    pclass=parcels.JITParticle,
    lon=np.random.uniform(*lon_bds, size=(number_particles, )),
    lat=np.random.uniform(*lat_bds, size=(number_particles, )),
    time=time_0 + np.zeros(shape=(number_particles, ))
)

In [32]:
def CheckError(particle, fieldset, time):
    if particle.state >= 50:  # This captures all Errors
        particle.delete()

In [33]:
output_file = pset.ParticleFile(
    name="Parcels_FESOM6.zarr", outputdt=datetime.timedelta(hours=1)
)

#we execute the particle file name
pset.execute(
    [parcels.AdvectionRK4, CheckError],
    runtime=datetime.timedelta(days=90),
    dt=datetime.timedelta(minutes=10),
    output_file=output_file,
)

INFO: Output files are stored in Parcels_FESOM6.zarr.


INFO:parcels.tools.loggers:Output files are stored in Parcels_FESOM6.zarr.


100%|██████████████████████| 7776000.0/7776000.0 [01:40<00:00, 77626.29it/s]


In [34]:
ds = xr.open_zarr("Parcels_FESOM6.zarr").compute()
ds

<xarray.Dataset> Size: 3kB
Dimensions:     (trajectory: 100, obs: 1)
Coordinates:
  * obs         (obs) int32 4B 0
  * trajectory  (trajectory) int64 800B 1000 1001 1002 1003 ... 1097 1098 1099
Data variables:
    lat         (trajectory, obs) float32 400B 5.95 12.77 8.838 ... 8.562 9.205
    lon         (trajectory, obs) float32 400B 4.241 2.688 ... 0.3862 1.944
    time        (trajectory, obs) timedelta64[ns] 800B 00:00:00 ... 00:00:00
    z           (trajectory, obs) float32 400B 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_kernels:        JITParticleAdvectionRK4CheckError
    parcels_mesh:           flat
    parcels_version:        3.0.5